In [3]:
import requests
import pandas as pd
from time import sleep

def get_address(postcode: str, huisnummer: int):
    url = f"https://api.bag.kadaster.nl/lvbag/individuelebevragingen/v2/adressen?postcode={postcode}&huisnummer={huisnummer}&exacteMatch=false&page=1&pageSize=20&inclusiefEindStatus=true"
    headers = {
    'X-Api-Key': 'l7380e3cfc716f4788beea66f144e155d9'
    }

    response = requests.request("GET", url, headers=headers).json()
    sleep(1/50)

    straat = response["_embedded"]["adressen"][0]["korteNaam"]
    huis = response["_embedded"]["adressen"][0]["huisnummer"]
    plaats = response["_embedded"]["adressen"][0]["woonplaatsNaam"]

    return straat,huis,plaats


def get_address_toevoeging(postcode: str, huisnummer: int, toevoeging: str):
    if toevoeging:
        url = f"https://api.bag.kadaster.nl/lvbag/individuelebevragingen/v2/adressen?postcode={postcode}&huisnummer={huisnummer}&huisletter={toevoeging}&exacteMatch=false&page=1&pageSize=20&inclusiefEindStatus=true"
    
    headers = {
    'X-Api-Key': 'l7380e3cfc716f4788beea66f144e155d9'
    }

    response = requests.request("GET", url, headers=headers).json()
    sleep(1/50)

    straat = response["_embedded"]["adressen"][0]["korteNaam"]
    huis = response["_embedded"]["adressen"][0]["huisnummer"]
    huisletter = response["_embedded"]["adressen"][0]["huisletter"]
    plaats = response["_embedded"]["adressen"][0]["woonplaatsNaam"]

    return straat,huis,huisletter,plaats


def convert_postcodes(dataframe: pd.DataFrame):
    """Assumes a dataframe of 3 columns. postcode -> huisnummer -> huisletter"""
    straten = []
    huisnummers = []
    toevoegingen = []
    plaatsen = []

    for row in range(len(dataframe)):
        letter = dataframe.iloc[row,2]
        if letter:
            straat,huis,huisletter,plaats = get_address_toevoeging(dataframe.iloc[row,0], dataframe.iloc[row,1], letter)

            straten.append(straat)
            huisnummers.append(huis)
            toevoegingen.append(huisletter)
            plaatsen.append(plaats)


        else:
            straat,huis,plaats = get_address(dataframe.iloc[row,0], dataframe.iloc[row,1])

            straten.append(straat)
            huisnummers.append(huis)
            toevoegingen.append(None)
            plaatsen.append(plaats)

    data = {"straat": straten,
            "huisnummer": huisnummers,
            "huisletter": toevoegingen,
            "plaats": plaatsen}
    return pd.DataFrame(data)

In [4]:
sample_data = {"postcode": ["3111PB", "3124TK", "2512JL"],
               "huisnummer": [116,13,177],
               "huisletetr": [None, None, "b"]}


df = pd.DataFrame(sample_data)
print(df)

  postcode  huisnummer huisletetr
0   3111PB         116       None
1   3124TK          13       None
2   2512JL         177          b


In [1]:
df2 = convert_postcodes(df)
print(df2)

NameError: name 'convert_postcodes' is not defined

In [9]:
df_t = pd.read_csv("test_map.csv", sep=";")
len(df_t.columns)

3

In [6]:
df_t

,postcode;huisnummer;letter
0,2512 JL;177;B
1,1223 JA;186;
2,5663tg;36;
3,8085 BT;109;
4,5768rw;68;
5,2042 dx;13;
6,2512jl;173;u


               straat  huisnummer           stad
0         Groenweegje         116       Schiedam
1  's-Gravenzandelaan         177  's-Gravenhage
2  's-Gravenzandelaan         177  's-Gravenhage


               straat  huisnummer           stad huisletter
0         Groenweegje         116       Schiedam       None
1  's-Gravenzandelaan         177  's-Gravenhage          B
2  's-Gravenzandelaan         177  's-Gravenhage       None
